In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer

path_to_llm_weights = "/home/mielliott/llama/hf/7B"

tokenizer = LlamaTokenizer.from_pretrained(path_to_llm_weights)
model = LlamaForCausalLM.from_pretrained(path_to_llm_weights, device_map="auto")
model.tie_weights()

/home/mielliott/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.35s/it]


In [2]:
def run(q, **kwargs):
    if type(q) == str:
        q = [q]

    input_ids = tokenizer(q, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, return_dict_in_generate=True, **kwargs) # output_scores=True
    return str(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))

In [10]:
print(run("User: Who is the current president of the University of Florida?\nAI Chat Bot: He is named ", max_new_tokens=100))

User: Who is the current president of the University of Florida?
AI Chat Bot: His name is 100% correct.
User: What is the name of the current president of the University of Florida?
AI Chat Bot: His name is 100% correct. His name is 100% correct. His name is 100% correct. His name is 100% correct. His name is 100% correct. His name is 100% correct. His name is 100% correct. His


Turn raw label transcriptions into DwC records

In [52]:
label = """
0 1 2 3 4 5 6 7 8 9 10		
cm	copyright reserved	
The New York
Botanical Garden


WI
288


PLANTS OF BAHAMA ISLANDS

Buchneria floridana Gand.

Grand Bahama: in wet soil about Pelican Lake.

Flowers lavender-pink.
May 24, 1975

D. S. Correll & John Popenoe 45411

Herbarium
FAIRCHILD TROPICAL GARDEN


NEW YORK
BOTANICAL
GARDEN


NEW YORK BOTANICAL GARDEN
01334304


NY IMAGED
"""

In [122]:
question = "Here is the same information as a Darwin Core record:"
record_primer = """{
    """
record = record_primer

In [134]:
test_terms = """
occurrenceID
catalogNumber
recordedBy
year
month
day
country
stateProvince
locality
decimalLatitude
decimalLongitude
scientificName
kingdom
phylum
class
order
family
genus
specificEpithet
infraspecificEpithet
identifiedBy
basisOfRecord
institutionCode
collectionCode
institutionName
rights
"""

term_gen = (term for term in test_terms.split())

In [135]:
print(record)

for next_term in term_gen:
    record += f'"{next_term}": "'
    prompt = f'{label}\n\n{question}\n\n{record}'
    # new_value = "haha"
    result = run(prompt, max_new_tokens=100)[len(prompt):]
    new_value = result[:result.find('"')]
    record = f"{record}{new_value}\",\n    "
    print(record.split("\n")[-2])

{
    "occurrenceID": "01334304",
    "catalogNumber": "01334304",
    "recordedBy": "NYBG",
    "year": "2014",
    "month": "05",
    "day": "24",
    "country": "US",
    "stateProvince": "NY",
    "locality": "Grand Bahama: in wet soil about Pelican Lake.",
    "decimalLatitude": "25.000000",
    "decimalLongitude": "77.000000",
    
    "occurrenceID": "01334304",
    "catalogNumber": "01334304",
    "recordedBy": "NYBG",
    "year": "2014",
    "month": "05",
    "day": "24",
    "country": "US",
    "stateProvince": "NY",
    "locality": "Grand Bahama: in wet soil about Pelican Lake.",
    "decimalLatitude": "25.000000",
    "decimalLongitude": "77.000000",
    "scientificName": "Buchneria floridana Gand.",
    "kingdom": "Plantae",
    "phylum": "Tracheophyta",
    "class": "Magnoliopsida",
    "order": "Asterales",
    "family": "Asteraceae",
    "genus": "Buchneria",
    "specificEpithet": "floridana Gand.",
    "infraspecificEpithet": "Gand.",
    "identifiedBy": "D. S. Corr

In [137]:
print(record)

{
    "occurrenceID": "01334304",
    "catalogNumber": "01334304",
    "recordedBy": "NYBG",
    "year": "2014",
    "month": "05",
    "day": "24",
    "country": "US",
    "stateProvince": "NY",
    "locality": "Grand Bahama: in wet soil about Pelican Lake.",
    "decimalLatitude": "25.000000",
    "decimalLongitude": "77.000000",
    "occurrenceID": "01334304",
    "catalogNumber": "01334304",
    "recordedBy": "NYBG",
    "year": "2014",
    "month": "05",
    "day": "24",
    "country": "US",
    "stateProvince": "NY",
    "locality": "Grand Bahama: in wet soil about Pelican Lake.",
    "decimalLatitude": "25.000000",
    "decimalLongitude": "77.000000",
    "scientificName": "Buchneria floridana Gand.",
    "kingdom": "Plantae",
    "phylum": "Tracheophyta",
    "class": "Magnoliopsida",
    "order": "Asterales",
    "family": "Asteraceae",
    "genus": "Buchneria",
    "specificEpithet": "floridana Gand.",
    "infraspecificEpithet": "Gand.",
    "identifiedBy": "D. S. Correll &